In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
meteodata = pd.read_parquet('../data/raw/historical_meteorological_information_expanded.parquet')

In [3]:
meteodata['time'] = pd.to_datetime(meteodata['time'])
meteodata = meteodata[meteodata.time > pd.to_datetime('2015-01-01')]
meteodata = meteodata.replace(-9999, None)
meteodata['sunrise'] = pd.to_datetime(meteodata['sunrise'])
meteodata['sunset'] = pd.to_datetime(meteodata['sunset'])
meteodata['day_duration'] = (meteodata['sunset']-meteodata['sunrise']).dt.seconds
meteodata['temperature_range'] = meteodata['temperature_2m_max'] - meteodata['temperature_2m_min']  
meteodata['temperature_max_apparent_range'] = meteodata['temperature_2m_max'] - meteodata['apparent_temperature_max']  
meteodata['temperature_min_apparent_range'] = meteodata['temperature_2m_min'] - meteodata['apparent_temperature_min']  
meteodata['temperature_mean_apparent_range'] = meteodata['temperature_2m_mean'] - meteodata['apparent_temperature_mean'] 
meteodata = meteodata.dropna()
meteodata.drop(['sunrise', 'sunset'], axis=1, inplace=True)

In [4]:
meteodata['new_time'] = meteodata['time'] + datetime.timedelta(days=10)

In [55]:
# This should be done in  a groupby modality:
meteodata_for_cve = meteodata[meteodata.CVE_CONCATENADA=='01001']

def get_daily_information_rolling_window(meteodata_for_cve):
    days=90
    drop_cols = ['CVE_CONCATENADA', 'time']
    rolling = meteodata_for_cve.drop(drop_cols, axis=1).rolling(f'{days}D', on='new_time', min_periods=days, closed='left')
    daily_information_rolling_info = {}
    daily_information_rolling_info['mean'] = rolling.mean()
    daily_information_rolling_info['std'] = rolling.std()
    daily_information_rolling_info['max'] = rolling.max()
    daily_information_rolling_info['min'] = rolling.min()
    daily_information_rolling_info['median'] = rolling.median()
    daily_information_rolling_info['skew'] = rolling.skew()
    daily_information_rolling_info['kurt'] = rolling.kurt()
    daily_information_rolling_info['mean_vs_median'] = daily_information_rolling_info['mean'] - daily_information_rolling_info['median']
    daily_information_rolling_info['range'] = daily_information_rolling_info['max'] - daily_information_rolling_info['min']
    return daily_information_rolling_info

In [ ]:
rolling_meteodata = meteodata.groupby('CVE_CONCATENADA').apply(get_daily_information_rolling_window)

,time,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,shortwave_radiation_sum,precipitation_sum,rain_sum,precipitation_hours,CVE_CONCATENADA,elevation,day_duration,temperature_range,temperature_max_apparent_range,temperature_min_apparent_range,temperature_mean_apparent_range,new_time
12785,2015-01-02,21.5,10.7,16.5,18.7,9.0,13.3,15.0,0.0,0.0,0.0,01001,1865.0,39180,10.8,2.8,1.7,3.2,2015-01-12
12786,2015-01-03,20.5,9.6,15.1,15.7,7.4,11.7,16.85,0.3,0.3,3.0,01001,1865.0,39180,10.9,4.8,2.2,3.4,2015-01-13
12787,2015-01-04,20.8,9.2,14.8,17.4,6.7,12.1,10.94,0.2,0.2,1.0,01001,1865.0,39180,11.6,3.4,2.5,2.7,2015-01-14
12788,2015-01-05,16.2,6.4,10.3,11.6,1.6,5.5,12.24,0.0,0.0,0.0,01001,1865.0,39240,9.8,4.6,4.8,4.8,2015-01-15
12789,2015-01-06,21.5,7.8,12.5,18.3,4.3,8.9,14.67,0.0,0.0,0.0,01001,1865.0,39300,13.7,3.2,3.5,3.6,2015-01-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15865,2023-06-09,33.0,13.0,23.1,32.2,10.5,20.9,30.3,0.0,0.0,0.0,32058,2090.0,48480,20.0,0.8,2.5,2.2,2023-06-19
15866,2023-06-10,33.1,14.6,24.4,30.9,13.0,22.1,30.5,0.0,0.0,0.0,32058,2090.0,48480,18.5,2.2,1.6,2.3,2023-06-20
15867,2023-06-11,34.3,13.7,24.4,33.7,11.4,22.1,30.34,0.0,0.0,0.0,32058,2090.0,48420,20.6,0.6,2.3,2.3,2023-06-21
15868,2023-06-12,34.2,15.2,25.0,33.8,14.8,23.9,28.67,0.0,0.0,0.0,32058,2090.0,48480,19.0,0.4,0.4,1.1,2023-06-22
